<a href="https://colab.research.google.com/github/christinaxliu/research/blob/main/HZExoplanetClassifiers/KNNClassifier/HZExoplanet_KNN_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pickle
from google.colab import drive

In [25]:
warnings.filterwarnings('ignore')
pd.options.mode.copy_on_write = True
plt.style.use('fivethirtyeight')

In [26]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
exoplanets_filename = '/content/drive/My Drive/Colab Notebooks/HZExoplanetClassifiers/KNNClassifier/KeplerConfirmedExoplanets_2024.06.17.csv'
exoplanets_data = pd.read_table(exoplanets_filename,
                                skiprows=98,
                                sep=',',
                                header=None,
                                index_col=None,
                                names = ['pl_name', 'hostname', 'default_flag', 'sy_snum', 'sy_pnum', 'discoverymethod', 'disc_year', 'disc_facility',
                                          'pl_orbper', 'pl_orbpererr1', 'pl_orbpererr2', 'pl_orbsmax', 'pl_orbsmaxerr1', 'pl_orbsmaxerr2',
                                          'pl_rade', 'pl_radeerr1', 'pl_radeerr2', 'pl_radj', 'pl_radjerr1', 'pl_radjerr2',
                                          'pl_masse', 'pl_masseerr1', 'pl_masseerr2', 'pl_massj', 'pl_massjerr1', 'pl_massjerr2',
                                          'pl_dens', 'pl_denserr1', 'pl_denserr2', 'pl_orbeccen', 'pl_orbeccenerr1', 'pl_orbeccenerr2',
                                          'pl_insol', 'pl_insolerr1', 'pl_insolerr2', 'pl_eqt', 'pl_eqterr1', 'pl_eqterr2',
                                          'pl_ratdor', 'pl_ratdorerr1', 'pl_ratdorerr2', 'pl_ratror', 'pl_ratrorerr1', 'pl_ratrorerr2',
                                          'st_spectype', 'st_teff', 'st_tefferr1', 'st_tefferr2', 'st_rad', 'st_raderr1', 'st_raderr2',
                                          'st_mass', 'st_masserr1', 'st_masserr2', 'st_met', 'st_meterr1', 'st_meterr2',
                                          'st_metratio', 'st_lum', 'st_lumerr1', 'st_lumerr2', 'st_logg', 'st_loggerr1', 'st_loggerr2',
                                          'st_age', 'st_ageerr1', 'st_ageerr2', 'st_dens', 'st_denserr1', 'st_denserr2', 'st_vsin',
                                          'st_vsinerr1', 'st_vsinerr2', 'st_rotp', 'st_rotperr1', 'st_rotperr2', 'st_radv', 'st_radverr1',
                                          'st_radverr2', 'sy_dist', 'sy_disterr1', 'sy_disterr2', 'sy_bmag', 'sy_bmagerr1', 'sy_bmagerr2', 'sy_vmag',
                                          'sy_vmagerr1', 'sy_vmagerr2'],
                                skipfooter=0,
                                engine='python')

In [28]:
# Sanity check the first several rows of data to ensure data is read correctly
exoplanets_data.head()

,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,pl_orbper,pl_orbpererr1,...,st_radverr2,sy_dist,sy_disterr1,sy_disterr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2
0,2MASS J19383260+4603591 b,2MASS J19383260+4603591,1,2,3,Eclipse Timing Variations,2015,Kepler,406.000000,4.000000,...,NaN,396.332,6.3885,-6.3885,12.112,0.307,-0.307,12.651,0.215,-0.215
1,KIC 10001893 b,KIC 10001893,1,1,3,Orbital Brightness Modulation,2014,Kepler,0.219700,NaN,...,NaN,1672.940,149.7000,-149.7000,15.073,0.109,-0.109,15.829,0.150,-0.150
2,KIC 10001893 c,KIC 10001893,1,1,3,Orbital Brightness Modulation,2014,Kepler,0.325280,NaN,...,NaN,1672.940,149.7000,-149.7000,15.073,0.109,-0.109,15.829,0.150,-0.150
3,KIC 10001893 d,KIC 10001893,1,1,3,Orbital Brightness Modulation,2014,Kepler,0.811610,NaN,...,NaN,1672.940,149.7000,-149.7000,15.073,0.109,-0.109,15.829,0.150,-0.150
4,KIC 10068024 b,KIC 10068024,1,1,1,Orbital Brightness Modulation,2021,Kepler,2.073549,0.000008,...,NaN,721.105,NaN,NaN,13.685,0.087,-0.087,13.144,0.092,-0.092


In [29]:
# Sanity check the last several rows of data to ensure data is read correctly.
exoplanets_data.tail()

,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,pl_orbper,pl_orbpererr1,...,st_radverr2,sy_dist,sy_disterr1,sy_disterr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2
2769,Kepler-997 b,Kepler-997,1,1,1,Transit,2016,Kepler,2.707307,0.000006,...,NaN,467.568,5.707,-5.574,12.794,0.312,-0.312,11.521,0.038,-0.038
2770,Kepler-998 b,Kepler-998,1,1,1,Transit,2016,Kepler,5.653777,0.000011,...,NaN,986.364,15.666,-15.197,15.304,0.023,-0.023,14.463,0.080,-0.080
2771,Kepler-999 b,Kepler-999,1,1,1,Transit,2016,Kepler,5.991857,0.000008,...,NaN,545.063,8.197,-7.962,13.045,0.016,-0.016,12.364,0.069,-0.069
2772,PH1 b,PH1,1,4,1,Transit,2012,Kepler,138.317000,0.040000,...,-0.2,1033.160,NaN,NaN,14.932,0.067,-0.067,13.841,0.149,-0.149
2773,PH2 b,PH2,1,1,1,Transit,2013,Kepler,282.525420,0.000110,...,NaN,343.863,3.631,-3.557,13.446,0.023,-0.023,12.645,0.046,-0.046


In [30]:
# Get the statistics about the data
exoplanets_data.describe()

,default_flag,sy_snum,sy_pnum,disc_year,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,...,st_radverr2,sy_dist,sy_disterr1,sy_disterr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2
count,2774.0,2774.000000,2774.000000,2774.000000,2774.000000,2330.000000,2330.000000,1004.000000,225.000000,225.000000,...,53.000000,2697.000000,2605.000000,2605.000000,2738.000000,2738.000000,2738.000000,2774.000000,2774.00000,2774.00000
mean,1.0,1.044701,1.876352,2015.966474,32.409955,1.233449,-0.327138,0.170554,0.029845,-0.021249,...,-0.221698,822.496410,24.935287,-23.305747,15.352788,0.073115,-0.073115,14.528330,0.13487,-0.13487
std,0.0,0.223451,1.176245,2.632557,90.774635,50.365528,14.517035,0.260871,0.167735,0.080998,...,0.347672,449.530121,49.152204,40.184245,1.422158,0.073592,0.073592,1.292512,0.13205,0.13205
min,1.0,1.000000,1.000000,2009.000000,0.219700,0.000000,-700.000000,0.006000,0.000120,-0.900000,...,-1.100000,36.439600,0.039500,-1073.720000,8.431000,0.000000,-0.575000,8.250000,0.00200,-1.13300
25%,1.0,1.000000,1.000000,2014.000000,5.145515,0.000013,-0.000185,0.062000,0.001000,-0.010000,...,-0.200000,489.876000,5.349000,-26.955000,14.560250,0.031000,-0.070000,13.752000,0.08000,-0.14900
50%,1.0,1.000000,1.000000,2016.000000,11.456085,0.000049,-0.000049,0.103000,0.002700,-0.002700,...,-0.100000,771.494000,12.838000,-12.677000,15.522000,0.064000,-0.064000,14.793500,0.10300,-0.10300
75%,1.0,1.000000,2.000000,2016.000000,27.079978,0.000184,-0.000013,0.181000,0.010000,-0.001000,...,-0.010000,1047.030000,27.690000,-5.282500,16.308000,0.070000,-0.031000,15.489000,0.14900,-0.08000
max,1.0,4.000000,8.000000,2023.000000,2500.000000,2400.000000,0.000000,4.200000,2.400000,-0.000130,...,0.000000,3460.510000,1507.570000,-0.039400,19.340000,0.575000,0.000000,18.190000,1.13300,-0.00200


In [31]:
exoplanets_data['sy_snum'] = exoplanets_data['sy_snum'].astype(int)
exoplanets_data['sy_pnum'] = exoplanets_data['sy_pnum'].astype(int)
exoplanets_data['disc_year'] = exoplanets_data['disc_year'].astype(int)
exoplanets_data['pl_orbper'] = exoplanets_data['pl_orbper'].astype(float)
exoplanets_data['pl_orbpererr1'] = exoplanets_data['pl_orbpererr1'].astype(float)
exoplanets_data['pl_orbpererr2'] = exoplanets_data['pl_orbpererr2'].astype(float)
exoplanets_data['pl_orbsmax'] = exoplanets_data['pl_orbsmax'].astype(float)
exoplanets_data['pl_orbsmaxerr1'] = exoplanets_data['pl_orbsmaxerr1'].astype(float)
exoplanets_data['pl_orbsmaxerr2'] = exoplanets_data['pl_orbsmaxerr2'].astype(float)
exoplanets_data['pl_rade'] = exoplanets_data['pl_rade'].astype(float)
exoplanets_data['pl_radeerr1'] = exoplanets_data['pl_radeerr1'].astype(float)
exoplanets_data['pl_radeerr2'] = exoplanets_data['pl_radeerr2'].astype(float)
exoplanets_data['pl_radj'] = exoplanets_data['pl_radj'].astype(float)
exoplanets_data['pl_radjerr1'] = exoplanets_data['pl_radjerr1'].astype(float)
exoplanets_data['pl_radjerr2'] = exoplanets_data['pl_radjerr2'].astype(float)
exoplanets_data['pl_masse'] = exoplanets_data['pl_masse'].astype(float)
exoplanets_data['pl_masseerr1'] = exoplanets_data['pl_masseerr1'].astype(float)
exoplanets_data['pl_masseerr2'] = exoplanets_data['pl_masseerr2'].astype(float)
exoplanets_data['pl_massj'] = exoplanets_data['pl_massj'].astype(float)
exoplanets_data['pl_massjerr1'] = exoplanets_data['pl_massjerr1'].astype(float)
exoplanets_data['pl_massjerr2'] = exoplanets_data['pl_massjerr2'].astype(float)
exoplanets_data['pl_dens'] = exoplanets_data['pl_dens'].astype(float)
exoplanets_data['pl_denserr1'] = exoplanets_data['pl_denserr1'].astype(float)
exoplanets_data['pl_denserr2'] = exoplanets_data['pl_denserr2'].astype(float)
exoplanets_data['pl_orbeccen'] = exoplanets_data['pl_orbeccen'].astype(float)
exoplanets_data['pl_orbeccenerr1'] = exoplanets_data['pl_orbeccenerr1'].astype(float)
exoplanets_data['pl_orbeccenerr2'] = exoplanets_data['pl_orbeccenerr2'].astype(float)
exoplanets_data['pl_insol'] = exoplanets_data['pl_insol'].astype(float)
exoplanets_data['pl_insolerr1'] = exoplanets_data['pl_insolerr1'].astype(float)
exoplanets_data['pl_insolerr2'] = exoplanets_data['pl_insolerr2'].astype(float)
exoplanets_data['pl_eqt'] = exoplanets_data['pl_eqt'].astype(float)
exoplanets_data['pl_eqterr1'] = exoplanets_data['pl_eqterr1'].astype(float)
exoplanets_data['pl_eqterr2'] = exoplanets_data['pl_eqterr2'].astype(float)
exoplanets_data['pl_ratdor'] = exoplanets_data['pl_ratdor'].astype(float)
exoplanets_data['pl_ratdorerr1'] = exoplanets_data['pl_ratdorerr1'].astype(float)
exoplanets_data['pl_ratdorerr2'] = exoplanets_data['pl_ratdorerr2'].astype(float)
exoplanets_data['pl_ratror'] = exoplanets_data['pl_ratror'].astype(float)
exoplanets_data['pl_ratrorerr1'] = exoplanets_data['pl_ratrorerr1'].astype(float)
exoplanets_data['pl_ratrorerr2'] = exoplanets_data['pl_ratrorerr2'].astype(float)
exoplanets_data['st_spectype'] = exoplanets_data['st_spectype'].astype(str)
exoplanets_data['st_teff'] = exoplanets_data['st_teff'].astype(float)
exoplanets_data['st_tefferr1'] = exoplanets_data['st_tefferr1'].astype(float)
exoplanets_data['st_tefferr2'] = exoplanets_data['st_tefferr2'].astype(float)
exoplanets_data['st_rad'] = exoplanets_data['st_rad'].astype(float)
exoplanets_data['st_raderr1'] = exoplanets_data['st_raderr1'].astype(float)
exoplanets_data['st_raderr2'] = exoplanets_data['st_raderr2'].astype(float)
exoplanets_data['st_mass'] = exoplanets_data['st_mass'].astype(float)
exoplanets_data['st_masserr1'] = exoplanets_data['st_masserr1'].astype(float)
exoplanets_data['st_masserr2'] = exoplanets_data['st_masserr2'].astype(float)
exoplanets_data['st_met'] = exoplanets_data['st_met'].astype(float)
exoplanets_data['st_meterr1'] = exoplanets_data['st_meterr1'].astype(float)
exoplanets_data['st_meterr2'] = exoplanets_data['st_meterr2'].astype(float)
exoplanets_data['st_lum'] = exoplanets_data['st_lum'].astype(float)
exoplanets_data['st_lumerr1'] = exoplanets_data['st_lumerr1'].astype(float)
exoplanets_data['st_lumerr2'] = exoplanets_data['st_lumerr2'].astype(float)
exoplanets_data['st_logg'] = exoplanets_data['st_logg'].astype(float)
exoplanets_data['st_loggerr1'] = exoplanets_data['st_loggerr1'].astype(float)
exoplanets_data['st_loggerr2'] = exoplanets_data['st_loggerr2'].astype(float)
exoplanets_data['st_age'] = exoplanets_data['st_age'].astype(float)
exoplanets_data['st_ageerr1'] = exoplanets_data['st_ageerr1'].astype(float)
exoplanets_data['st_ageerr2'] = exoplanets_data['st_ageerr2'].astype(float)
exoplanets_data['st_dens'] = exoplanets_data['st_dens'].astype(float)
exoplanets_data['st_denserr1'] = exoplanets_data['st_denserr1'].astype(float)
exoplanets_data['st_denserr2'] = exoplanets_data['st_denserr2'].astype(float)
exoplanets_data['st_vsin'] = exoplanets_data['st_vsin'].astype(float)
exoplanets_data['st_vsinerr1'] = exoplanets_data['st_vsinerr1'].astype(float)
exoplanets_data['st_vsinerr2'] = exoplanets_data['st_vsinerr2'].astype(float)
exoplanets_data['st_rotp'] = exoplanets_data['st_rotp'].astype(float)
exoplanets_data['st_rotperr1'] = exoplanets_data['st_rotperr1'].astype(float)
exoplanets_data['st_rotperr2'] = exoplanets_data['st_rotperr2'].astype(float)
exoplanets_data['st_radv'] = exoplanets_data['st_radv'].astype(float)
exoplanets_data['st_radverr1'] = exoplanets_data['st_radverr1'].astype(float)
exoplanets_data['st_radverr2'] = exoplanets_data['st_radverr2'].astype(float)
exoplanets_data['sy_dist'] = exoplanets_data['sy_dist'].astype(float)
exoplanets_data['sy_disterr1'] = exoplanets_data['sy_disterr1'].astype(float)
exoplanets_data['sy_disterr2'] = exoplanets_data['sy_disterr2'].astype(float)
exoplanets_data['sy_bmag'] = exoplanets_data['sy_bmag'].astype(float)
exoplanets_data['sy_bmagerr1'] = exoplanets_data['sy_bmagerr1'].astype(float)
exoplanets_data['sy_bmagerr2'] = exoplanets_data['sy_bmagerr2'].astype(float)
exoplanets_data['sy_vmag'] = exoplanets_data['sy_vmag'].astype(float)
exoplanets_data['sy_vmagerr1'] = exoplanets_data['sy_vmagerr1'].astype(float)
exoplanets_data['sy_vmagerr2'] = exoplanets_data['sy_vmagerr2'].astype(float)

In [32]:
# Change any fields with empty spaces to NaN values so we could clean them up later on
exoplanets_data = exoplanets_data.applymap(lambda x: np.nan if isinstance(x, str) and x.isspace() else x)

In [33]:
exoplanets_data['pl_eqt'].describe()

count     213.000000
mean      883.272300
std       493.351927
min       134.000000
25%       478.000000
50%       798.000000
75%      1173.000000
max      2550.000000
Name: pl_eqt, dtype: float64

In [34]:
exoplanets_data.loc[((~np.isnan(exoplanets_data['pl_eqt'])) & (exoplanets_data['pl_eqt'] > 180) & (exoplanets_data['pl_eqt'] < 310)), 'label'] = 1
exoplanets_data.loc[((~np.isnan(exoplanets_data['pl_eqt'])) & (exoplanets_data['pl_eqt'] <= 180) | (exoplanets_data['pl_eqt'] >= 310)), 'label'] = 0

In [35]:
exoplanets_data['label'].value_counts()

label
0.0    193
1.0     20
Name: count, dtype: int64

In [36]:
# Drop off data row without label
training_data = exoplanets_data.loc[~np.isnan(exoplanets_data['label'])]
training_data

,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,pl_orbper,pl_orbpererr1,...,sy_dist,sy_disterr1,sy_disterr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,label
15,KOI-1257 b,KOI-1257,1,2,1,Transit,2014,Kepler,86.647661,3.400000e-05,...,NaN,NaN,NaN,15.830,0.098,-0.098,14.818,0.069,-0.069,0.0
16,KOI-13 b,KOI-13,1,3,1,Transit,2011,Kepler,1.763588,1.000000e-06,...,519.0960,30.7030,-27.5060,10.654,0.078,-0.078,9.791,0.023,-0.023,0.0
23,KOI-1831 d,Kepler-324,1,1,4,Transit,2021,Kepler,34.169000,NaN,...,501.5740,3.6810,-3.6290,15.762,0.068,-0.068,14.222,0.069,-0.069,0.0
24,KOI-1833 d,Kepler-968,1,1,3,Transit,2021,Kepler,7.685000,NaN,...,290.3240,1.4480,-1.4350,16.175,0.064,-0.064,14.626,0.126,-0.126,0.0
27,KOI-3503 b,KOI-3503,1,1,2,Transit,2021,Kepler,21.185000,NaN,...,887.0640,12.3700,-12.3700,14.556,0.042,-0.042,13.831,0.114,-0.114,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,Kepler-93 b,Kepler-93,1,1,2,Transit,2014,Kepler,4.726740,9.700000e-07,...,95.9115,0.2217,-0.2206,10.731,0.062,-0.062,9.996,0.004,-0.004,0.0
2710,Kepler-949 c,Kepler-949,1,1,2,Transit,2023,Kepler,20.997580,3.334000e-05,...,486.1050,5.3870,-5.3870,15.720,0.031,-0.031,14.718,0.172,-0.172,0.0
2744,Kepler-975 c,Kepler-975,1,1,2,Transit,2023,Kepler,5.058213,4.740000e-06,...,426.0880,3.5640,-3.5640,16.118,0.065,-0.065,14.633,0.046,-0.046,0.0
2772,PH1 b,PH1,1,4,1,Transit,2012,Kepler,138.317000,4.000000e-02,...,1033.1600,NaN,NaN,14.932,0.067,-0.067,13.841,0.149,-0.149,0.0


In [37]:
# Drop data fields that are not relevant to training or don't provide value to training
training_data = training_data.drop(['pl_name'], axis = 1)
training_data = training_data.drop(['hostname'], axis = 1)
training_data = training_data.drop(['default_flag'], axis = 1)
training_data = training_data.drop(['discoverymethod'], axis = 1)
training_data = training_data.drop(['disc_year'], axis = 1)
training_data = training_data.drop(['disc_facility'], axis = 1)
training_data = training_data.drop(['pl_orbpererr1'], axis = 1)
training_data = training_data.drop(['pl_orbpererr2'], axis = 1)
training_data = training_data.drop(['pl_orbsmaxerr1'], axis = 1)
training_data = training_data.drop(['pl_orbsmaxerr2'], axis = 1)
training_data = training_data.drop(['pl_radeerr1'], axis = 1)
training_data = training_data.drop(['pl_radeerr2'], axis = 1)
training_data = training_data.drop(['pl_radjerr1'], axis = 1)
training_data = training_data.drop(['pl_radjerr2'], axis = 1)
training_data = training_data.drop(['pl_masseerr1'], axis = 1)
training_data = training_data.drop(['pl_masseerr2'], axis = 1)
training_data = training_data.drop(['pl_massjerr1'], axis = 1)
training_data = training_data.drop(['pl_massjerr2'], axis = 1)
training_data = training_data.drop(['pl_denserr1'], axis = 1)
training_data = training_data.drop(['pl_denserr2'], axis = 1)
training_data = training_data.drop(['pl_orbeccenerr1'], axis = 1)
training_data = training_data.drop(['pl_orbeccenerr2'], axis = 1)
training_data = training_data.drop(['pl_insolerr1'], axis = 1)
training_data = training_data.drop(['pl_insolerr2'], axis = 1)
training_data = training_data.drop(['pl_eqterr1'], axis = 1)
training_data = training_data.drop(['pl_eqterr2'], axis = 1)
training_data = training_data.drop(['pl_ratdorerr1'], axis = 1)
training_data = training_data.drop(['pl_ratdorerr2'], axis = 1)
training_data = training_data.drop(['pl_ratrorerr1'], axis = 1)
training_data = training_data.drop(['pl_ratrorerr2'], axis = 1)
training_data = training_data.drop(['st_spectype'], axis = 1)
training_data = training_data.drop(['st_tefferr1'], axis = 1)
training_data = training_data.drop(['st_tefferr2'], axis = 1)
training_data = training_data.drop(['st_raderr1'], axis = 1)
training_data = training_data.drop(['st_raderr2'], axis = 1)
training_data = training_data.drop(['st_masserr1'], axis = 1)
training_data = training_data.drop(['st_masserr2'], axis = 1)
training_data = training_data.drop(['st_meterr1'], axis = 1)
training_data = training_data.drop(['st_meterr2'], axis = 1)
training_data = training_data.drop(['st_metratio'], axis = 1)
training_data = training_data.drop(['st_lumerr1'], axis = 1)
training_data = training_data.drop(['st_lumerr2'], axis = 1)
training_data = training_data.drop(['st_loggerr1'], axis = 1)
training_data = training_data.drop(['st_loggerr2'], axis = 1)
training_data = training_data.drop(['st_ageerr1'], axis = 1)
training_data = training_data.drop(['st_ageerr2'], axis = 1)
training_data = training_data.drop(['st_denserr1'], axis = 1)
training_data = training_data.drop(['st_denserr2'], axis = 1)
training_data = training_data.drop(['st_vsinerr1'], axis = 1)
training_data = training_data.drop(['st_vsinerr2'], axis = 1)
training_data = training_data.drop(['st_rotperr1'], axis = 1)
training_data = training_data.drop(['st_rotperr2'], axis = 1)
training_data = training_data.drop(['st_radverr1'], axis = 1)
training_data = training_data.drop(['st_radverr2'], axis = 1)
training_data = training_data.drop(['sy_disterr1'], axis = 1)
training_data = training_data.drop(['sy_disterr2'], axis = 1)
training_data = training_data.drop(['sy_bmagerr1'], axis = 1)
training_data = training_data.drop(['sy_bmagerr2'], axis = 1)
training_data = training_data.drop(['sy_vmagerr1'], axis = 1)
training_data = training_data.drop(['sy_vmagerr2'], axis = 1)
training_data.to_csv('/content/drive/My Drive/Colab Notebooks/HZExoplanetClassifiers/KNNClassifier/TrainingData_DropInrelevantFields_2024.06.17.csv')

# Drop data fields that are with too many missing values
training_data = training_data.drop(['st_lum'], axis = 1)
training_data = training_data.drop(['st_vsin'], axis = 1)
training_data = training_data.drop(['st_rotp'], axis = 1)
training_data = training_data.drop(['st_radv'], axis = 1)
training_data.to_csv('/content/drive/My Drive/Colab Notebooks/HZExoplanetClassifiers/KNNClassifier/TrainingData_Final_2024.06.17.csv')

training_data

,sy_snum,sy_pnum,pl_orbper,pl_orbsmax,pl_rade,pl_radj,pl_masse,pl_massj,pl_dens,pl_orbeccen,...,st_rad,st_mass,st_met,st_logg,st_age,st_dens,sy_dist,sy_bmag,sy_vmag,label
15,2,1,86.647661,0.38200,10.540,0.940,460.83000,1.45000,2.10,0.77200,...,1.13,0.99,0.270,4.32,9.3,0.9900,NaN,15.830,14.818,0.0
16,3,1,1.763588,0.03641,16.948,1.512,2949.46240,9.28000,3.60,0.00064,...,1.74,1.72,0.200,4.20,NaN,NaN,519.0960,10.654,9.791,0.0
23,1,4,34.169000,NaN,1.130,0.101,2.23000,0.00702,9.95,NaN,...,0.83,0.90,NaN,NaN,NaN,NaN,501.5740,15.762,14.222,0.0
24,1,3,7.685000,NaN,2.580,0.230,11.02000,0.03467,4.50,NaN,...,0.67,0.68,NaN,NaN,NaN,NaN,290.3240,16.175,14.626,0.0
27,1,2,21.185000,NaN,1.190,0.106,9.20000,0.02895,10.00,NaN,...,1.25,0.86,NaN,NaN,NaN,NaN,887.0640,14.556,13.831,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,1,2,4.726740,0.05343,1.478,0.132,4.66000,0.01466,7.93,0.00000,...,0.92,0.91,-0.180,NaN,6.6,NaN,95.9115,10.731,9.996,0.0
2710,1,2,20.997580,0.13940,2.970,NaN,NaN,NaN,NaN,0.00000,...,0.90,0.82,0.080,4.45,NaN,11.7201,486.1050,15.720,14.718,0.0
2744,1,2,5.058213,0.05670,10.690,NaN,NaN,NaN,NaN,0.00000,...,1.67,0.95,0.560,3.97,NaN,1.1333,426.0880,16.118,14.633,0.0
2772,4,1,138.317000,0.65200,6.180,0.551,169.00000,0.53100,NaN,0.07020,...,1.73,1.53,0.210,4.14,2.0,0.2930,1033.1600,14.932,13.841,0.0


In [38]:
# Apply simple imputation by filling in missing values with mean
training_data = training_data.fillna(training_data.mean())
training_data = training_data.dropna()
training_data

,sy_snum,sy_pnum,pl_orbper,pl_orbsmax,pl_rade,pl_radj,pl_masse,pl_massj,pl_dens,pl_orbeccen,...,st_rad,st_mass,st_met,st_logg,st_age,st_dens,sy_dist,sy_bmag,sy_vmag,label
15,2,1,86.647661,0.382000,10.540,0.940000,460.830000,1.450000,2.100000,0.772000,...,1.13,0.99,0.270000,4.320000,9.300000,0.990000,650.655365,15.830,14.818,0.0
16,3,1,1.763588,0.036410,16.948,1.512000,2949.462400,9.280000,3.600000,0.000640,...,1.74,1.72,0.200000,4.200000,4.944743,2.030714,519.096000,10.654,9.791,0.0
23,1,4,34.169000,0.226208,1.130,0.101000,2.230000,0.007020,9.950000,0.091273,...,0.83,0.90,0.035114,4.374451,4.944743,2.030714,501.574000,15.762,14.222,0.0
24,1,3,7.685000,0.226208,2.580,0.230000,11.020000,0.034670,4.500000,0.091273,...,0.67,0.68,0.035114,4.374451,4.944743,2.030714,290.324000,16.175,14.626,0.0
27,1,2,21.185000,0.226208,1.190,0.106000,9.200000,0.028950,10.000000,0.091273,...,1.25,0.86,0.035114,4.374451,4.944743,2.030714,887.064000,14.556,13.831,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,1,2,4.726740,0.053430,1.478,0.132000,4.660000,0.014660,7.930000,0.000000,...,0.92,0.91,-0.180000,4.374451,6.600000,2.030714,95.911500,10.731,9.996,0.0
2710,1,2,20.997580,0.139400,2.970,0.501626,301.073354,0.947293,7.447455,0.000000,...,0.90,0.82,0.080000,4.450000,4.944743,11.720100,486.105000,15.720,14.718,0.0
2744,1,2,5.058213,0.056700,10.690,0.501626,301.073354,0.947293,7.447455,0.000000,...,1.67,0.95,0.560000,3.970000,4.944743,1.133300,426.088000,16.118,14.633,0.0
2772,4,1,138.317000,0.652000,6.180,0.551000,169.000000,0.531000,7.447455,0.070200,...,1.73,1.53,0.210000,4.140000,2.000000,0.293000,1033.160000,14.932,13.841,0.0


In [39]:
features = training_data.drop(['label'], axis = 1)
results = training_data.label

In [40]:
# Split data with 70% for training and 30% for testing
features_train, features_test, results_train, results_test = train_test_split(features, results, test_size = 0.3, random_state = 0)

In [41]:
# Standadize the scales of features
sc = StandardScaler()
features_train_sc = sc.fit_transform(features_train)
features_test_sc = sc.transform(features_test)

In [42]:
# Train a KNN classifier with the training data
knn_classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn_classifier.fit(features_train_sc, results_train)

KNeighborsClassifier()

In [43]:
# Call trained KNN classifier to get predictions for the testing data
results_pred = knn_classifier.predict(features_test_sc)

In [44]:
# Calculate accuracy, precision, recall, and F-1 scores for the KNN classifier
print("KNN Classifier Accuracy: ", accuracy_score(results_test, results_pred))
print()
print("KNN Classifier Classification Report :\n",(classification_report(results_test,results_pred)))

KNN Classifier Accuracy:  0.921875

KNN Classifier Classification Report :
               precision    recall  f1-score   support

         0.0       0.93      0.98      0.96        58
         1.0       0.67      0.33      0.44         6

    accuracy                           0.92        64
   macro avg       0.80      0.66      0.70        64
weighted avg       0.91      0.92      0.91        64



In [45]:
# Save the KNN classifier model onto disk
knnPickle = open('/content/drive/My Drive/Colab Notebooks/HZExoplanetClassifiers/KNNClassifier/hz_exoplanet_knn_classifier_v1', 'wb')
pickle.dump(knn_classifier, knnPickle)
knnPickle.close()

In [46]:
# Load the KNN classifier model from disk and test with test data
loaded_knn_classifier = pickle.load(open('/content/drive/My Drive/Colab Notebooks/HZExoplanetClassifiers/KNNClassifier/hz_exoplanet_knn_classifier_v1', 'rb'))
pred_results = loaded_knn_classifier.predict(features_test_sc)
pred_results

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])